# Tarea 3 - Reconocimiento de patrones en Minería de datos

## Integrantes

|Nombre | Rol|
|:------|:---|
|Felipe Avaria| 2923547-3|
|Rolando Casanueva| 201204505-3|


## Desarrollo

### Información previa

- Descripción de Apache Spark y MLlib

**Apache spark** es una herramienta rápida y genérica para el procesamientos de datos a gran escala *Big Data*. Provee una **API** de alto nivel en distintos lenguages, tales como *Java*, *Scala*, *Python* y *R* y además incluye un sistema optimizado para ejecuciones de gráficos. También soporta herramientas como SQL tanto en estructura como procesamiento, a la vez, posee MLlib para máquinas de aprendizaje, GraphX para procesamiento de gráficos y Spark Streaming. Por otra parte **MLlib** como se mencionó, está enfocado en máquinas de aprendizaje. Esta contiene una gran variedad de algoritmos para clasificación, regresión, recomendación, clustering entre otros. Y como en esta tarea realizaremos un sistema recomendador, podemos notar como la seccions de recomendación que incluye *ALS* (Alternating least squares) está presente, será nuestra guía. [Documentación PySpark MLlib](https://spark.apache.org/docs/latest/api/python/pyspark.mllib.html)



- ¿Qué son los RDD y Dataframe?

**RDD** (Resilient Distributed Dataset) es una abstracción básica de Spark, la cual representa una colección de elementos inmutables y particionados que puede ser operados de manera paralela. Mientras que **Dataframe**, es una colección de elementos agrupados bajo nombres de columnas. Este últimos se maneja mediantes un *SQLContext* dadas las caracteristicas del mismo. Ambos son agrupaciones de elementos pero sus estructuras los diferencian.


- Descripción del dataset utilizado.

Como se menciona en el readme inicial del repositorio. El dataset puede ser descargado en [este link](https://grouplens.org/datasets/movielens/10m/) y dado que utilizaremos un método de *Machine Learning*, es conveniente dividir el archivo `rating`, para esto deberemos correr
```
c:\PATH\TO\DATASET\ bash split_ratings.sh
```

#### Formato

La estructura de los archivos son las siguiente, *notar que los archivos divididos de rating tendrán el mismo formato que el original*

| Dataset Name | Structure |
| :----------- | :-------- |
| RATINGS      |`UserID::MovieID::Rating::Timestamp` |
| TAGS         |`UserID::MovieID::Tag::Timestamp`    |
| MOVIES       |`MovieID::Title::Genres`             |


#### Descripciones
|Parámetro | Descripción|
|:-|:-|
|UserID | número identificador del usuario. |
|MovieID | número identificador de la película. |
|Rating | calificación 1 a 5 entregada por el usuario a la película. |
|Timestamp | valor númerico del tiempo cuando se realizo la calificación.|
|Title | titulo de la pelicula. |
|Genres| genero de la pelicula|

| Nombres de | Generos |  |
| :------------ |:-|:-|
|Action|Adventure|Animation|
|Children's|Comedy|Crime|
|Documentary|Drama|Fantasy|
|Film-Noir|Horror|Musical|
|Mystery|Romance|Sci-Fi|
|Thriller|War|Western|


----------------------

### Análisis de resultados

Nosotros utilizamos **Python - PySpark**, de lo que utilizamos una carga de datos **RDD**. Luego por medio del algoritmo *ALS*, entrenamos con los conjuntos de entrenamiento `rX.train` donde `x = [1, 2, 3, 4, 5]` estos entrenamientos fueron con 10 configuraciones distintas variando `rank` y `lambda_`

```python
ranks = [1, 5, 10, 50, 100]
lambdas = [0.01, 0.02]
model.train(data, rank, lambda_)
```

**Notese:** 

    rank:      Rank of the feature matrices computed (number of features).
    lambda:    Regularization parameter.

Luego a estos modelos se les solicitó predecir mediante el conjunto de valores `rX.test`. De donde los valores predichos se comparan con los reales y se genera un **RMSE** (Error  cuadrático medio) ([más info](https://es.wikipedia.org/wiki/Error_cuadr%C3%A1tico_medio)).

\begin{equation}
RMSE = \sqrt{\frac{1}{n} \sum_{i=1}^{n}{(Y_i - Y_i')^2}}
\end{equation}

Así se obtuvieron los siguientes resultados

|rank|lambda|RMSE|
|:---|:-----|:---|
|1|0.01|0.823726942384|
|1|0.02|0.823562791225|
|5|0.01|0.791236482234|
|5|0.02|0.793231245956|
|10|0.01|0.713129758221|
|10|0.02|0.731251212394|
|50|0.01|0.623109532812|
|50|0.02|0.649788721923|
|100|0.01|0.323125098472|
|100|0.02|0.423129748299|

De estos resultados podemos desprender que si bien en `ranks` pequeños aumentar el lambda beneficia el aprendizaje, es completamente perjudicial el aumento de lambda a medida que los ranks van aumentando, esto se debe a que al poseer una gran variedad de caracteristicas las matrices tienden a moverse más en cuanto a los pesos lineales de regulación. Es por esto que a medida que más grande sea el valor de regulación más dificil es acertar al valor exacto porque el ajuste puede sobrepasar el valor **OVERFITTING**.

-------------------------

### Rendimiento

Desde la documentación de Spark se espera que el rendimiento al aumentar la memoria RAM sea espectacular, ya que se reducen las escrituras en disco, al trabajar de forma distribuida, cada proceso usara una cantidad de memoria muy alta, pero se podra trabajar con todos los procesadores y se aprovechara de mejor manera el tiempo de proceso al no tener que esperar la carga de los datos del disco.

De los resutados no se llega a ver un gran cambio en el rendimiento, esto se puede explicar dado que el tamaño de todo el dataset, considerando las tres tablas es de cerca de 60MB estaticos que luego al usar RDD deberian usar entre 5 y 7 veces el tamaño, por lo que se tendria  un tamaño esperado de 420MB con lo que el uso de 4gb no seria necesario, si se ve una mejora, en tiempo, lo que puede deberse a que por la profundidad del árbol este factor no sea como el encontrado en la documentación y en las referncias encontradas, si no que tendria que ser mas profundo en este caso particular.